In [8]:
import random
import copy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from reservoirpy import datasets
from reservoirpy.nodes import Reservoir, Ridge
import networkx as nx
import reservoirpy as rpy
rpy.verbosity(0)

0

In [9]:
n_timesteps = 2000
X = datasets.mackey_glass(n_timesteps=n_timesteps, sample_len=2000)
Y = np.roll(X, -1)

train_end = int(len(X) * 0.7)
test_start = train_end + 1
X_train, Y_train = X[:train_end], Y[:train_end]
X_test, Y_test = X[test_start:], Y[test_start:]
X_train = X_train.reshape(-1, 1)  # Reshape to (n_samples, 1) if X_train was a flat array


# Example call (Ensure correct values for population_size and genome_length)
population_size = 50
genome_length = 100
n_generations = 20

genome_template = {
    'input_dim': 1,  # Default value
    'output_dim': 1,  # Default value
    'reservoir_size': 100,  # Default value
    'spectral_radius': 0.95,  # Default value
    'leak_rate': 0.2,  # Default value
    'input_scaling': 0.5,  # Default value
    'bias_scaling': 0.5,  # Default value
    # Add other necessary parameters here
}


In [10]:
def initialize_genome(length):
    """Initializes a genome with a list of random parameters."""
    return {'input_dim': 1, 'output_dim': 1, 
            'reservoir_size': random.randint(50, 200),
            'spectral_radius': random.uniform(0.5, 1.5),
            'leak_rate': random.uniform(0.1, 1.0),
            'input_scaling': random.uniform(0.1, 0.9),
            'bias_scaling': random.uniform(0.1, 0.9)}

# Generate an initial population of genomes
def generate_initial_population(population_size, genome_initializer):
    return [genome_initializer(genome_length) for _ in range(population_size)]


def select_best_genomes(population, fitness_scores, top_k=10):
    sorted_genomes = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)
    return [genome for genome, _ in sorted_genomes[:top_k]]

def plot_metrics(metrics, title):
    generations = list(range(len(metrics['RMSE'])))
    plt.figure(figsize=(12, 8))
    plt.plot(generations, metrics['RMSE'], label='RMSE')
    plt.plot(generations, metrics['MAE'], label='MAE')
    plt.plot(generations, metrics['MSE'], label='MSE')
    plt.title(title)
    plt.xlabel('Generation')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()

In [19]:
def hox_mutation(genome):
    if len(genome) < 2:
        return genome  # Not enough elements to duplicate
    
    # Convert dictionary keys to a list and pick a random segment to 'duplicate'
    keys = list(genome.keys())
    start = random.randint(0, len(keys) // 2)
    end = random.randint(start + 1, len(keys))
    segment_keys = keys[start:end]
    
    # For simplicity, we could mutate the values of the keys in the segment
    # This is not a direct analog to duplicating a segment in a list but demonstrates
    # modifying a 'segment' of parameters within a dictionary.
    for key in segment_keys:
        # Mutate the parameter slightly
        # This mutation logic will depend on the parameter; this is a placeholder
        if isinstance(genome[key], int):
            genome[key] += random.randint(-10, 10)  # Example mutation for integer parameters
        elif isinstance(genome[key], float):
            genome[key] += random.uniform(-0.1, 0.1)  # Example mutation for float parameters
    
    return genome



def regular_mutation(genome):
    """
    Applies regular mutation to the genome. Mutates a parameter value with some probability.
    """
    # Choose a random parameter to mutate
    param_to_mutate = random.choice(list(genome.keys()))

    # Example mutation process for numeric parameters
    if param_to_mutate in ['reservoir_size', 'input_dim', 'output_dim', 'spectral_radius', 'leak_rate', 'input_scaling', 'bias_scaling']:
        # Apply a mutation by modifying the parameter value slightly
        # This is just an example; the actual mutation logic will depend on the parameter and model needs
        if isinstance(genome[param_to_mutate], int):
            genome[param_to_mutate] += random.randint(-10, 10)  # For integer parameters
        else:
            genome[param_to_mutate] += random.uniform(-0.1, 0.1)  # For floating-point parameters

        # Ensure mutated values remain within logical bounds (example)
        if param_to_mutate == 'spectral_radius':
            genome[param_to_mutate] = max(0.1, min(genome[param_to_mutate], 1.5))
        elif param_to_mutate == 'leak_rate':
            genome[param_to_mutate] = max(0.1, min(genome[param_to_mutate], 1.0))
        # Add similar bounds checks for other parameters as necessary

    # Handle non-numeric parameters or special cases if necessary

    return genome



def mutate_genome(genome):
    """Mutates the genome, with a chance to apply HOX mutations."""
    if random.random() < 0.1:  # Arbitrary probability to apply HOX mutation
        return hox_mutation(genome)
    else:
        # Apply regular mutation
        return regular_mutation(genome)  # This function needs to be defined.
    
def mutate_genome1(genome):
    """Applies mutations to a genome."""
    # This is a placeholder for mutation logic. You might randomly alter parameters.
    mutated_genome = copy.deepcopy(genome)
    if random.random() < 0.1:  # Example mutation rate
        mutated_genome['reservoir_size'] = random.randint(50, 200)  # Example mutation
    return mutated_genome

def reproduce_genomes(selected_genomes, population_size):
    """Generates a new population through reproduction and mutation."""
    new_population = []
    while len(new_population) < population_size:
        parent = random.choice(selected_genomes)
        offspring = mutate_genome(parent)
        new_population.append(offspring)
    return new_population


In [14]:
def transcribe_genome_to_model_config(genome):
    # Initialize a configuration dictionary with default values
    model_config = {
        'input_dim': 1,  # Default input dimension
        'output_dim': 1,  # Default output dimension
        'reservoir_size': 100,  # Default reservoir size
        'spectral_radius': 0.95,  # Default spectral radius
        'leak_rate': 0.2,  # Default leak rate
        'input_scaling': 0.5,  # Default input scaling
        'bias_scaling': 0.5,  # Default bias scaling
    }

    # Update the model configuration with values from the genome
    # Only update if the parameter exists in the genome
    for param, value in genome.items():
        if param in model_config:
            model_config[param] = value

    # Ensure 'input_dim' matches your actual input data's feature dimension
    # If your data has a different number of features, adjust 'input_dim' accordingly
    # For example, if your data has 2 features, you should set 'input_dim' to 2
    model_config['input_dim'] = 2  # Uncomment and adjust as necessary based on your data

    return model_config




def setup_model(model_config):
    model = Reservoir(units=model_config['reservoir_size'],
                    sr=model_config['spectral_radius'],
                    Win=np.random.uniform(-model_config['input_scaling'], model_config['input_scaling'], 
                    (model_config['reservoir_size'], model_config['input_dim'])),
                    bias=np.random.uniform(-model_config['bias_scaling'], model_config['bias_scaling'], (model_config['reservoir_size'], 1)))
    readout = Ridge(output_dim=model_config['output_dim'])
    esn = model >> readout
    return esn

def evaluate_fitness(genome, X_train, Y_train, X_test, Y_test):
    model_config = transcribe_genome_to_model_config(genome)
    model = setup_model(model_config)
    
    # Train the model and generate predictions
    model.fit(X_train, Y_train)
    predictions = model.run(X_test)
    
    rmse, mae, mse = evaluate_performance(Y_test, predictions)
    return rmse, mae, mse

def evaluate_performance(real, predicted):
    rmse = np.sqrt(mean_squared_error(real, predicted))
    mae = mean_absolute_error(real, predicted)
    mse = mean_squared_error(real, predicted)
    return rmse, mae, mse

In [20]:
def run_evolution(population_size, genome_length, n_generations, X_train, Y_train, X_test, Y_test):
    population = generate_initial_population(population_size, initialize_genome)
    metrics = {'RMSE': [], 'MAE': [], 'MSE': []}
    
    for generation in range(n_generations):
        fitness_scores = [evaluate_fitness(genome, X_train, Y_train, X_test, Y_test) for genome in population]
        metrics['RMSE'].append(np.mean([fs[0] for fs in fitness_scores]))
        metrics['MAE'].append(np.mean([fs[1] for fs in fitness_scores]))
        metrics['MSE'].append(np.mean([fs[2] for fs in fitness_scores]))
        
        best_genomes = select_best_genomes(population, fitness_scores)
        population = reproduce_genomes(best_genomes, population_size)
        
        print(f"Generation {generation}: Best RMSE: {min(metrics['RMSE'])}, Best MAE: {min(metrics['MAE'])}, Best MSE: {min(metrics['MSE'])}")
    
    plot_metrics(metrics, "Evolution of Performance Metrics")

run_evolution(population_size, genome_length, n_generations, X_train, Y_train, X_test, Y_test)

/Users/chaitravshetty/Downloads/Advanced-Genomes-for-Evolutionary-Computing-main 3/hierarchical_genomes/.venv/lib/python3.10/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=2.28475e-22): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")
/Users/chaitravshetty/Downloads/Advanced-Genomes-for-Evolutionary-Computing-main 3/hierarchical_genomes/.venv/lib/python3.10/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=4.79718e-21): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")
/Users/chaitravshetty/Downloads/Advanced-Genomes-for-Evolutionary-Computing-main 3/hierarchical_genomes/.venv/lib/python3.10/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=7.68691e-20): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")
/Users/chaitravshetty/Downlo

Generation 0: Best RMSE: 1.243909235414727, Best MAE: 0.1749402445208309, Best MSE: 72.44536270728013


/Users/chaitravshetty/Downloads/Advanced-Genomes-for-Evolutionary-Computing-main 3/hierarchical_genomes/.venv/lib/python3.10/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=5.87242e-21): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")


ValueError: Ridge-152is  expecting data of shape (-6,) but received shape (1,).